In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.decomposition import PCA

# `Load extracted feature maps`

In [ ]:
X_train = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/train_feature_maps.npy')
X_test = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/test_feature_maps.npy')
X_val = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/val_feature_maps.npy')
y_train = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/train_labels.npy')
y_test = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/test_labels.npy')
y_val = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/val_labels.npy')

# `Convert one-hot encoded labels to class indices`

In [ ]:
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
y_val = np.argmax(y_val, axis=1)

In [ ]:
# Number of classes
num_classes = len(np.unique(y_train))

# Reshape 4D feature maps to 2D (samples, features)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)
X_val_reshaped = X_val.reshape(X_val.shape[0], -1)




# `Apply PCA for dimensionality reduction`

In [ ]:
pca = PCA(n_components=500)
X_train_pca = pca.fit_transform(X_train_reshaped)
X_test_pca = pca.transform(X_test_reshaped)
X_val_pca = pca.transform(X_val_reshaped)

In [ ]:
# Save PCA model
joblib.dump(pca, "pca.pkl")

['pca.pkl']

# `Standardize features`

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_pca)
X_test_scaled = scaler.transform(X_test_pca)
X_val_scaled = scaler.transform(X_val_pca)

In [ ]:
# Save Scaler
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

# `Define MLP model`

In [ ]:
model = Sequential([
    Dense(1024, activation='relu', input_shape=(500,)),  # Use PCA reduced shape
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(256, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax')  # Corrected output layer
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for integer labels
              metrics=['accuracy'])

In [8]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

In [9]:
# Train the model
history = model.fit(X_train_scaled, y_train, 
                    epochs=100, batch_size=32,
                    validation_data=(X_val_scaled, y_val),
                    callbacks=[early_stopping, lr_scheduler],
                    verbose=1)

Epoch 1/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 16s 14ms/step - accuracy: 0.0157 - loss: 6.3465 - val_accuracy: 0.1869 - val_loss: 4.2137 - learning_rate: 0.0010
Epoch 2/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.1654 - loss: 4.2116 - val_accuracy: 0.3450 - val_loss: 3.1051 - learning_rate: 0.0010
Epoch 3/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.3040 - loss: 3.2353 - val_accuracy: 0.4174 - val_loss: 2.7159 - learning_rate: 0.0010
Epoch 4/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.3959 - loss: 2.6960 - val_accuracy: 0.4538 - val_loss: 2.5227 - learning_rate: 0.0010
Epoch 5/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.4388 - loss: 2.4191 - val_accuracy: 0.4743 - val_loss: 2.4086 - learning_rate: 0.0010
Epoch 6/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.4823 - loss: 2.1792 - val_accuracy: 0.4948 - val_loss: 2.3107 - learning_rate: 0.0010
Epoch 7/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.5

# `Save trained model`

In [ ]:
model.save("mlp.h5")

# `Testing performance`

In [11]:
# Predictions for each dataset
y_train_pred = np.argmax(model.predict(X_train_scaled), axis=1)
y_val_pred = np.argmax(model.predict(X_val_scaled), axis=1)
y_test_pred = np.argmax(model.predict(X_test_scaled), axis=1)

998/998 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [12]:
# Compute accuracy for train, validation, and test sets
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [13]:
# Print accuracies
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Training Accuracy: 77.93%
Validation Accuracy: 56.09%
Test Accuracy: 56.72%
